In [3]:
import mne, mne_bids, HelperFunctions
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from mne_bids import (BIDSPath, read_raw_bids)
from autoreject import AutoReject


parameters = {
    # data
    "data_path": "F:/Masterthesis/Data/",
    "task": "N170",
    "preprocessing": "light",
    "n_subjects": 40,
    "reject_incorrect_responses": True,
    "input_window_samples": 256,
}

In [32]:
def load_raw(parameters, subject_id):
    """
    Loads a single subject from the ERP Core data, applies filtering and ICA, and returns a mne.Raw object.
    """
    with HelperFunctions.suppress_stdout():
        bids_root = parameters["data_path"]+"/"+parameters["task"]
        bids_path = BIDSPath(subject=subject_id, task=parameters["task"],
                             session=parameters["task"], datatype='eeg', 
                             suffix='eeg', root=bids_root)
        raw = read_raw_bids(bids_path)
        raw.load_data()
        HelperFunctions.read_annotations_core(bids_path,raw)
        raw.set_channel_types({'HEOG_left': 'eog', 'HEOG_right': 'eog', 'VEOG_lower': 'eog'})
        raw.set_montage('standard_1020',match_case=False)
        if parameters["preprocessing"] == "medium":
            raw.filter(0.5,40)
        if parameters["preprocessing"] == "heavy":
            raw.filter(0.5,40)
            ica, badComps = HelperFunctions.load_precomputed_ica(bids_root, subject_id,parameters["task"])
            HelperFunctions.add_ica_info(raw,ica)
            ica.apply(raw)
        raw = raw.resample(128)
    return raw

def epoch_raw(parameters, raw):
    """
    Takes a mne.Raw object, loads the correct events, and returns a mne.Epoch object.
    """
    with HelperFunctions.suppress_stdout():
        # get correct tmin, tmax, and event mapping per task
        custom_mapping, tmin, tmax = get_task_specifics(parameters)
        
        # shift annotations by lcd monitor delay
        if parameters["task"] != "MNE":
            raw.annotations.onset = raw.annotations.onset+.026
        
        # load events
        (events_from_annot, event_dict) = mne.events_from_annotations(raw, event_id=custom_mapping)
        
        if parameters["reject_incorrect_responses"] == True and parameters["task"] in ["N170", "N400", "N2pc", "P3"]:
        # only include events where response is in time and correct
            events_0, lag_0 = mne.event.define_target_events(events_from_annot, 0, 201, raw.info['sfreq'], 0, 0.8, 0, 999)
            events_1, lag_1 = mne.event.define_target_events(events_from_annot, 1, 201, raw.info['sfreq'], 0, 0.8, 1, 999)
            events_from_annot = np.concatenate((events_0, events_1), axis=0)
            # sort event array by timepoints to get rid of warning
            events_from_annot = events_from_annot[events_from_annot[:, 0].argsort()]
            # drop responses
            event_dict.pop('response:201')
            event_dict.pop('response:202')           

        # epoch with no constant detrend to remove dc offset
        epoch = mne.Epochs(raw, events_from_annot, event_dict,
                           tmin=tmin,tmax=tmax, preload=True,
                           reject_by_annotation=True, baseline=None, 
                           picks=range(0,30), detrend=0)
        
        # apply autoreject for heavy preprocessing to remove artefacts
        if parameters["preprocessing"] == "heavy":
            ar = AutoReject()
            epoch = ar.fit_transform(epoch) 
    return epoch

def get_task_specifics(parameters):
    """
    Returns mapping, tmin, tmax, specific to the task.
    """
    tmin = -0.199
    tmax = 0.80
    if parameters["task"] == "N170":
        # Cars: 0, Faces: 1
        custom_mapping = dict(("stimulus:"+str(i), 1) for i in range(0,41))
        custom_mapping.update(dict(("stimulus:"+str(i), 0) for i in range(41,81)))
    elif parameters["task"] == "N400":
        # unrelated word: 0, Related word: 1
        custom_mapping = {'stimulus:221': 0, 'stimulus:222': 0, 
                          'stimulus:211': 1, 'stimulus:212': 1} 
    elif parameters["task"] == "P3":
        # target=stimulus (rare): 0, target!=stimulus (frequent): 1
        custom_mapping = {'stimulus:11': 0, 'stimulus:12': 1, 'stimulus:13': 1, 'stimulus:14': 1, 'stimulus:15': 1, 
                          'stimulus:21': 1, 'stimulus:22': 0, 'stimulus:23': 1, 'stimulus:24': 1, 'stimulus:25': 1,
                          'stimulus:31': 1, 'stimulus:32': 1, 'stimulus:33': 0, 'stimulus:34': 1, 'stimulus:35': 1,
                          'stimulus:41': 1, 'stimulus:42': 1, 'stimulus:43': 1, 'stimulus:44': 0, 'stimulus:45': 1,
                          'stimulus:51': 1, 'stimulus:52': 1, 'stimulus:53': 1, 'stimulus:54': 1, 'stimulus:55': 0}
    # add button responses to tasks that have them
    if parameters["reject_incorrect_responses"] == True and parameters["task"] in ["N170", "N400", "N2pc", "P3"]:
            custom_mapping.update({'response:201': 201, 'response:202': 202})
    return custom_mapping, tmin, tmax

def create_df(parameters):
    df_list = []
    for i in range(parameters["n_subjects"]):
        subjectID = f"{i+1:03d}"
        raw = load_raw(parameters, subjectID)
        epoch = epoch_raw(parameters, raw)
        df = epoch.to_data_frame()
        df["subjectID"] = i
        df_list.append(df)
    df = pd.concat(df_list, axis=0)
    
    # change condition naming to binary labels
    custom_mapping = get_task_specifics(parameters)[0]
    for condition in custom_mapping:
        df["condition"] = df["condition"].replace(condition,custom_mapping[condition])
    
    # reshape data
    data = df.iloc[:,3:33]
    data = data.to_numpy().reshape(int(data.shape[0]/parameters["input_window_samples"]), parameters["input_window_samples"], -1)
    data = np.transpose(data,axes=[0,2,1])
    # create labels
    df = df[["epoch","condition","subjectID"]].drop_duplicates()
    df = df.reset_index()
    
    df["data"]=pd.Series(list(data))
    df = df.drop(columns=["index"])
    return df

def load_df(parameters):
    df = pd.read_pickle(parameters["data_path"]+"/Dataframes/"+parameters["task"]+"_ds_"+parameters["preprocessing"]+".pkl")
    return df

def create_data_labels(df, list_of_subjects=None):
    """
    Takes dataframe and returns numpy versions of the data and labels. 
    """
    # get data from dataframe and reshape back
    if list_of_subjects != None:
        df = df[df["subjectID"].isin(list_of_subjects)]
    data = np.dstack(df["data"].to_numpy())
    data = np.moveaxis(data, -1, 0)
    # get labels from dataframe
    labels = df["condition"].to_numpy()
    
    return data, labels

In [33]:
raw = load_raw(parameters, "001")
e = epoch_raw(parameters,raw)
df = e.to_data_frame()

<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in EEG.data (sub-001_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-001_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Expected to find a single channels file associated with sub-001_ses-N170_task-N170, but found 2: "['F:\\Masterthesis\\Data\\N170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_badChannels.tsv', 'F:\\Masterthesis\\Data\\N170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_channels.tsv']".

The search_str was "F:\Masterthesis\Data\N170\sub-001\**\sub-001_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Participants file not found for sub-001_ses-N170_task-N170_eeg.set... Not reading in any particpants.tsv data.
  raw = read_raw_bids(bids_path)


In [34]:
df[df["epoch"]==0]

,time,condition,epoch,FP1,F3,F7,FC3,C3,C5,P3,...,FCz,Cz,C4,C6,P4,P8,P10,PO8,PO4,O2
0,-195,stimulus:80,0,45.596199,47.292836,46.466654,41.128503,31.878718,25.379012,10.135864,...,37.192617,28.406003,24.983277,43.754374,9.741328,31.474213,42.337803,20.595739,5.454449,8.279213
1,-188,stimulus:80,0,24.404889,28.461491,23.985574,25.265925,21.625795,13.511440,3.931843,...,23.355127,20.536652,22.640195,13.827365,9.125549,4.375870,14.574665,-5.512242,-1.311431,-6.946693
2,-180,stimulus:80,0,2.668258,6.948098,6.277180,6.864293,7.035689,-1.989608,-1.028976,...,4.150612,5.194571,3.556398,13.325658,-8.219313,-18.021922,-6.412509,-30.261847,-15.051173,-17.214696
3,-172,stimulus:80,0,-16.992458,-10.886024,-12.660524,-8.406941,-2.385155,-11.059883,-4.802285,...,-11.417976,-6.557006,-11.523773,-9.745009,-17.446089,-34.803403,-21.841026,-44.401731,-27.124281,-30.413412
4,-164,stimulus:80,0,-14.193366,-8.145872,-9.260839,-5.862750,-3.247439,-12.290643,-3.299425,...,-11.584371,-7.975423,-10.767908,-4.907827,-9.590886,-27.136627,-16.254731,-28.376640,-14.134598,-15.679527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,766,stimulus:80,0,8.635242,1.808964,13.996889,4.836501,2.124371,10.119360,6.139558,...,1.540429,0.460351,0.664874,0.746436,1.445564,2.187732,1.504510,-2.792277,-2.240896,-0.407900
124,773,stimulus:80,0,11.412126,2.824708,8.947058,3.818979,2.764923,10.180047,7.664329,...,2.512359,1.077859,-0.966865,-1.201587,1.387821,7.544793,7.358878,1.448891,-2.550330,0.591870
125,781,stimulus:80,0,22.382695,13.978760,22.814464,12.340680,7.434694,15.197139,7.333579,...,10.708669,7.163315,11.690747,21.156703,6.265479,15.546739,15.902253,5.051569,-0.814472,-0.218900
126,789,stimulus:80,0,20.038114,9.931568,16.959954,10.259957,6.698042,10.914976,6.760029,...,12.730795,12.037104,17.876320,20.325921,13.153664,25.667889,22.132428,16.895305,6.850399,9.174860


In [4]:
df = load_df(parameters)
data, labels = create_data_labels(df, [0,1,2])

In [35]:
# TODO: put this in a function
for task in ["N170", "N400", "P3"]:
    parameters["task"] = task
    for preprocessing in ["light", "medium", "heavy"]:
        parameters["preprocessing"] = preprocessing
        df = create_df(parameters)
        df.to_pickle(parameters["data_path"]+"/Dataframes/"+parameters["task"]+"_ds128_"+parameters["preprocessing"]+".pkl")

<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in EEG.data (sub-001_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-001_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Expected to find a single channels file associated with sub-001_ses-N170_task-N170, but found 2: "['F:\\Masterthesis\\Data\\N170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_badChannels.tsv', 'F:\\Masterthesis\\Data\\N170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_channels.tsv']".

The search_str was "F:\Masterthesis\Data\N170\sub-001\**\sub-001_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Participants file not found for sub-001_ses-N170_task-N170_eeg.set... Not reading in any particpants.tsv data.
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in E

<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in EEG.data (sub-010_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-010_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Expected to find a single channels file associated with sub-010_ses-N170_task-N170, but found 2: "['F:\\Masterthesis\\Data\\N170\\sub-010\\ses-N170\\eeg\\sub-010_ses-N170_task-N170_badChannels.tsv', 'F:\\Masterthesis\\Data\\N170\\sub-010\\ses-N170\\eeg\\sub-010_ses-N170_task-N170_channels.tsv']".

The search_str was "F:\Masterthesis\Data\N170\sub-010\**\sub-010_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Participants file not found for sub-010_ses-N170_task-N170_eeg.set... Not reading in any particpants.tsv data.
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in E

<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in EEG.data (sub-019_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-019_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Expected to find a single channels file associated with sub-019_ses-N170_task-N170, but found 2: "['F:\\Masterthesis\\Data\\N170\\sub-019\\ses-N170\\eeg\\sub-019_ses-N170_task-N170_badChannels.tsv', 'F:\\Masterthesis\\Data\\N170\\sub-019\\ses-N170\\eeg\\sub-019_ses-N170_task-N170_channels.tsv']".

The search_str was "F:\Masterthesis\Data\N170\sub-019\**\sub-019_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Participants file not found for sub-019_ses-N170_task-N170_eeg.set... Not reading in any particpants.tsv data.
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in E

<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in EEG.data (sub-028_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-028_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Expected to find a single channels file associated with sub-028_ses-N170_task-N170, but found 2: "['F:\\Masterthesis\\Data\\N170\\sub-028\\ses-N170\\eeg\\sub-028_ses-N170_task-N170_badChannels.tsv', 'F:\\Masterthesis\\Data\\N170\\sub-028\\ses-N170\\eeg\\sub-028_ses-N170_task-N170_channels.tsv']".

The search_str was "F:\Masterthesis\Data\N170\sub-028\**\sub-028_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Participants file not found for sub-028_ses-N170_task-N170_eeg.set... Not reading in any particpants.tsv data.
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in E

<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in EEG.data (sub-037_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-037_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Expected to find a single channels file associated with sub-037_ses-N170_task-N170, but found 2: "['F:\\Masterthesis\\Data\\N170\\sub-037\\ses-N170\\eeg\\sub-037_ses-N170_task-N170_badChannels.tsv', 'F:\\Masterthesis\\Data\\N170\\sub-037\\ses-N170\\eeg\\sub-037_ses-N170_task-N170_channels.tsv']".

The search_str was "F:\Masterthesis\Data\N170\sub-037\**\sub-037_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Participants file not found for sub-037_ses-N170_task-N170_eeg.set... Not reading in any particpants.tsv data.
  raw = read_raw_bids(bids_path)
<ipython-input-32-ed1cd67b1fc2>:10: RuntimeWarning: Data file name in E

...

Fold:  80%|█████████████████████████████████████████████████████████████▌               | 8/10 [00:00<00:00, 17.64it/s]


Fold: 100%|████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 17.71it/s]

n_interp:  33%|████████████████████████▋                                                 | 1/3 [00:01<00:02,  1.22s/it]

Repairing epochs:   0%|                                                                        | 0/146 [00:00<?, ?it/s]

Repairing epochs:  19%|███████████▉                                                  | 28/146 [00:00<00:00, 267.79it/s]

Repairing epochs:  38%|███████████████████████▎                                      | 55/146 [00:00<00:00, 245.42it/s]

Repairing epochs:  55%|█████████████████████████████████▉                            | 80/146 [00:00<00:00, 242.47it/s]

Repairing epochs:  72%|███████████████████████████████████████████▊                 | 105/146 [00:00<00:00, 238.72it/s]

Repairing epochs: 100%|████████

Repairing epochs: 100%|█████████████████████████████████████████████████████████████| 148/148 [00:00<00:00, 201.68it/s]



Fold:   0%|                                                                                     | 0/10 [00:00<?, ?it/s]


Fold:  20%|███████████████▍                                                             | 2/10 [00:00<00:00, 19.66it/s]


Fold:  40%|██████████████████████████████▊                                              | 4/10 [00:00<00:00, 19.55it/s]


Fold:  60%|██████████████████████████████████████████████▏                              | 6/10 [00:00<00:00, 19.51it/s]


Fold:  80%|█████████████████████████████████████████████████████████████▌               | 8/10 [00:00<00:00, 19.57it/s]


Fold: 100%|████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 19.51it/s]

n_interp:  67%|█████████████████████████████████████████████████▎                        | 2/3 [00:02<00:01,  1.32s/it]

Repairing epochs:   0%|  

Fold:  30%|███████████████████████                                                      | 3/10 [00:00<00:00, 20.46it/s]


Fold:  60%|██████████████████████████████████████████████▏                              | 6/10 [00:00<00:00, 20.30it/s]


Fold: 100%|████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 20.28it/s]

n_interp:  67%|█████████████████████████████████████████████████▎                        | 2/3 [00:02<00:01,  1.34s/it]

Repairing epochs:   0%|                                                                        | 0/140 [00:00<?, ?it/s]

Repairing epochs:  14%|████████▊                                                     | 20/140 [00:00<00:00, 195.88it/s]

Repairing epochs:  29%|█████████████████▋                                            | 40/140 [00:00<00:00, 182.55it/s]

Repairing epochs:  42%|██████████████████████████▏                                   | 59/140 [00:00<00:00, 182.05it/s]

Repairing epochs:  56%|███████

Repairing epochs:   0%|                                                                        | 0/142 [00:00<?, ?it/s]

Repairing epochs:  13%|████████▎                                                     | 19/142 [00:00<00:00, 188.57it/s]

Repairing epochs:  28%|█████████████████▍                                            | 40/142 [00:00<00:00, 196.28it/s]

Repairing epochs:  42%|██████████████████████████▏                                   | 60/142 [00:00<00:00, 192.97it/s]

Repairing epochs:  56%|██████████████████████████████████▉                           | 80/142 [00:00<00:00, 188.62it/s]

Repairing epochs:  70%|██████████████████████████████████████████▉                  | 100/142 [00:00<00:00, 189.40it/s]

Repairing epochs:  84%|███████████████████████████████████████████████████          | 119/142 [00:00<00:00, 188.83it/s]

Repairing epochs: 100%|█████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 189.16it/s]



Fold: 100%|███████████████████

ValueError: cannot reshape array of size 19787520 into shape (2576,256,newaxis)

In [79]:
import Training

parameters = {
    # data
    "data_path": "F:/Masterthesis/Data",
    "task": "N170",
    "preprocessing": "heavy",
    "n_subjects": 40,
    "reject_incorrect_responses": True,
    # model
    "model": "eegnet",
    "n_classes": 2,
    "n_chans": 30,
    "input_window_samples": 1025,
    "seed": 42,
    # classifier
    "lr": 0.001,
    "weight_decay": 0,
    "batch_size": 64,
    "n_epochs": 35,
    "n_splits": 5,
    "model_folder": "models_run3"
}


for model in ["eegnet", "shallow", "deep"]:
    parameters["model"] = model
    if model == "eegnet":
        parameters["lr"] = 0.001
    elif model == "shallow":
        parameters["lr"] = 0.000625
    elif model == "deep":
        parameters["lr"] = 0.01
        parameters["weight_decay"] = 0.0005
    for preprocessing in ["light", "medium", "heavy"]:
        parameters["preprocessing"] = preprocessing
        for task in ["N170", "N400", "P3"]:
            parameters["task"] = task
            parameters["task"] = task
            df = load_df(parameters)
            data, labels = create_data_labels(df)
            Training.run_exp(data, labels, parameters)

C:\Users\vapor\anaconda3\envs\braindecode\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 1 0 0] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Re-initializing optimizer because the following parameters were re-set: lr, weight_decay.
  epoch    train_accuracy    train_balanced_accuracy    train_loss    valid_accuracy    valid_balanced_accuracy    valid_loss      lr     dur
-------  ----------------  -------------------------  ------------  ----------------  -------------------------  ------------  ------  ------
      1            0.6965                     0.6979        0.6798            0.6716                     0.6730        0.6513  0.0010  6.0309
      2            0.7140                     0.7151        0.5948            0.7131                     0.7142        0.6165  0.0010  5.4694
      3            0.7406                     0.7411        0.5650            0.7174                     0.7179        0.5766  0.0010  5.5192
      4            0.7538                     0.7540        0.5442            0.7347                     0.7350        0.5573  0.0010  5.5202
      5            0.7523                     0.7533      

     10            0.7899                     0.7896        0.4880            0.7597                     0.7597        0.4921  0.0008  5.5901
     11            0.7953                     0.7950        0.4898            0.7623                     0.7621        0.4856  0.0008  5.6260
     12            0.7951                     0.7950        0.4801            0.7623                     0.7622        0.4808  0.0008  5.6289
     13            0.8057                     0.8056        0.4797            0.7649                     0.7648        0.4789  0.0007  5.6449
     14            0.8037                     0.8034        0.4708            0.7710                     0.7706        0.4758  0.0007  5.6459
     15            0.8063                     0.8066        0.4646            0.7744                     0.7750        0.4784  0.0006  5.6130
     16            0.8070                     0.8070        0.4677            0.7796                     0.7796        0.4787  0.0006  5.4893
     1

     22            0.8208                     0.8206        0.4357            0.7658                     0.7657        0.5023  0.0003  5.4983
     23            0.8214                     0.8213        0.4413            0.7649                     0.7649        0.4967  0.0003  5.5003
     24            0.8188                     0.8186        0.4381            0.7692                     0.7691        0.4995  0.0002  5.4973
     25            0.8208                     0.8206        0.4405            0.7675                     0.7674        0.5035  0.0002  5.4973
     26            0.8221                     0.8221        0.4366            0.7666                     0.7668        0.4982  0.0002  5.4993
     27            0.8206                     0.8202        0.4268            0.7684                     0.7682        0.5019  0.0001  5.4993
     28            0.8212                     0.8211        0.4375            0.7753                     0.7753        0.4988  0.0001  5.4983
     2

     34            0.8333                     0.8333        0.4243            0.7787                     0.7787        0.4885  0.0000  5.4584
     35            0.8335                     0.8335        0.4174            0.7787                     0.7787        0.4885  0.0000  5.4624
Re-initializing optimizer because the following parameters were re-set: lr, weight_decay.
  epoch    train_accuracy    train_balanced_accuracy    train_loss    valid_accuracy    valid_balanced_accuracy    valid_loss      lr     dur
-------  ----------------  -------------------------  ------------  ----------------  -------------------------  ------------  ------  ------
      1            0.7043                     0.7055        0.6737            0.6733                     0.6747        0.6452  0.0010  5.4813
      2            0.7255                     0.7257        0.5961            0.7156                     0.7162        0.6063  0.0010  5.4574
      3            0.7428                     0.7429      

...

C:\Users\vapor\anaconda3\envs\braindecode\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass classes=[0 1], y=[0 1 1 ... 1 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Re-initializing optimizer because the following parameters were re-set: lr, weight_decay.
  epoch    train_accuracy    train_balanced_accuracy    train_loss    valid_accuracy    valid_balanced_accuracy    valid_loss      lr     dur
-------  ----------------  -------------------------  ------------  ----------------  -------------------------  ------------  ------  ------
      1            0.5527                     0.6400        4.9797            0.5624                     0.6410        1.6108  0.0100  3.4270
      2            0.4942                     0.6315        1.8993            0.4996                     0.6378        1.2204  0.0100  2.3707
      3            0.7731                     0.6672        1.3197            0.7723                     0.6536        0.8574  0.0099  2.3399
      4            0.6649                     0.6733        1.1433            0.6538                     0.6415        1.0997  0.0098  2.3058
      5            0.7921                     0.6876      

     12            0.7934                     0.7131        0.5754            0.7862                     0.6822        0.7241  0.0076  2.1981
     13            0.7380                     0.7585        0.5710            0.7149                     0.6989        0.6410  0.0072  2.2011
     14            0.7857                     0.7767        0.5494            0.7676                     0.7294        0.6276  0.0068  2.2036
     15            0.8275                     0.7168        0.5542            0.8118                     0.6810        0.6660  0.0064  2.2101
     16            0.7281                     0.7763        0.5425            0.6956                     0.7122        0.6141  0.0059  2.2051
     17            0.7847                     0.7916        0.5151            0.7568                     0.7378        0.5963  0.0055  2.2111
     18            0.7946                     0.7670        0.5529            0.7645                     0.7141        0.6044  0.0050  2.2099
     1

     26            0.8068                     0.8118        0.4634            0.7676                     0.7478        0.5211  0.0016  2.3368
     27            0.8196                     0.8141        0.4570            0.7723                     0.7456        0.5219  0.0013  2.4446
     28            0.8107                     0.8070        0.4551            0.7614                     0.7423        0.5157  0.0010  2.3474
     29            0.8165                     0.8168        0.4491            0.7769                     0.7568        0.5166  0.0007  2.3373
     30            0.7868                     0.8159        0.4407            0.7452                     0.7592        0.5171  0.0005  2.3323
     31            0.8095                     0.8222        0.4474            0.7607                     0.7552        0.5120  0.0003  2.3254
     32            0.8043                     0.8215        0.4379            0.7552                     0.7519        0.5150  0.0002  2.3477
     3

      3            0.7581                     0.5949        1.4044            0.7514                     0.5789        1.1294  0.0099  2.2151
      4            0.7471                     0.6610        1.0595            0.7459                     0.6526        1.0676  0.0098  2.2181
      5            0.6585                     0.6903        0.9996            0.6468                     0.6674        0.8071  0.0097  2.2201
      6            0.7523                     0.6763        0.7279            0.7467                     0.6714        0.9108  0.0095  2.2180
      7            0.6779                     0.7088        0.7250            0.6731                     0.6951        0.6843  0.0093  2.2131
      8            0.8068                     0.6920        0.9317            0.8079                     0.7054        0.6488  0.0090  2.2176
      9            0.6686                     0.7195        0.6279            0.6638                     0.7112        0.6475  0.0087  2.2184
     1

In [6]:
results = []
for model in ["eegnet", "shallow", "deep"]:
    parameters["model"] = model
    for preprocessing in ["light", "medium", "heavy"]:
        parameters["preprocessing"] = preprocessing
        for task in ["N170", "N400", "P3"]:
            parameters["task"] = task
            df = Training.load_exp(parameters)
            results.append([model,preprocessing,task,df.loc[df.index[-1],"valid_balanced_accuracy"].mean()])
            #print(model+", "+preprocessing+", "+task+": "+str(df.loc[df.index[-1],"valid_balanced_accuracy"].mean()))
df_results = pd.DataFrame(results, columns=["Model", "Preprocessing", "Task", "Validation Balanced Accuracy"])

In [18]:
df_results[df_results["Preprocessing"]=="light"].pivot(
    index="Task", 
    columns="Model", 
    values="Validation Balanced Accuracy").style.highlight_max(color = 'lightgreen', axis = 1)

Model,deep,eegnet,shallow
Task,,,
N170,0.770606,0.777929,0.745493
N400,0.740646,0.744435,0.726546
P3,0.736983,0.738017,0.714750


In [8]:
df_results[df_results["Model"]=="eegnet"].pivot(
    index="Task", 
    columns="Preprocessing", 
    values="Validation Balanced Accuracy").style.highlight_max(color = 'lightgreen', axis = 1)

Preprocessing,heavy,light,medium
Task,,,
N170,0.769151,0.777929,0.772696
N400,0.734077,0.744435,0.745359
P3,0.753519,0.738017,0.743926


In [9]:
df_results[df_results["Model"]=="deep"].pivot(
    index="Task", 
    columns="Preprocessing", 
    values="Validation Balanced Accuracy").style.highlight_max(color = 'lightgreen', axis = 1)

Preprocessing,heavy,light,medium
Task,,,
N170,0.765149,0.770606,0.774588
N400,0.722008,0.740646,0.746593
P3,0.742308,0.736983,0.744185


In [10]:
df_results[df_results["Model"]=="shallow"].pivot(
    index="Task", 
    columns="Preprocessing", 
    values="Validation Balanced Accuracy").style.highlight_max(color = 'lightgreen', axis = 1)

Preprocessing,heavy,light,medium
Task,,,
N170,0.736216,0.745493,0.741383
N400,0.693768,0.726546,0.725070
P3,0.712448,0.714750,0.718637


In [70]:
parameters["preprocessing"] = "light"
raw = load_raw(parameters, "001")
epoch_light = epoch_raw(parameters, raw)

<ipython-input-59-03cda6a932f4>:7: RuntimeWarning: Data file name in EEG.data (sub-001_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-001_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-59-03cda6a932f4>:7: RuntimeWarning: Expected to find a single channels file associated with sub-001_ses-N170_task-N170, but found 2: "['F:\\Masterthesis\\Data\\N170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_badChannels.tsv', 'F:\\Masterthesis\\Data\\N170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_channels.tsv']".

The search_str was "F:\Masterthesis\Data\N170\sub-001\**\sub-001_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-59-03cda6a932f4>:7: RuntimeWarning: Participants file not found for sub-001_ses-N170_task-N170_eeg.set... Not reading in any particpants.tsv data.
  raw = read_raw_bids(bids_path)


In [71]:
parameters["preprocessing"] = "medium"
raw = load_raw(parameters, "001")
epoch_medium = epoch_raw(parameters, raw)

<ipython-input-59-03cda6a932f4>:7: RuntimeWarning: Data file name in EEG.data (sub-001_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-001_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-59-03cda6a932f4>:7: RuntimeWarning: Expected to find a single channels file associated with sub-001_ses-N170_task-N170, but found 2: "['F:\\Masterthesis\\Data\\N170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_badChannels.tsv', 'F:\\Masterthesis\\Data\\N170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_channels.tsv']".

The search_str was "F:\Masterthesis\Data\N170\sub-001\**\sub-001_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-59-03cda6a932f4>:7: RuntimeWarning: Participants file not found for sub-001_ses-N170_task-N170_eeg.set... Not reading in any particpants.tsv data.
  raw = read_raw_bids(bids_path)


In [69]:
parameters["preprocessing"] = "heavy"
raw = load_raw(parameters, "001")
epoch_heavy = epoch_raw(parameters, raw)

<ipython-input-59-03cda6a932f4>:7: RuntimeWarning: Data file name in EEG.data (sub-001_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-001_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-59-03cda6a932f4>:7: RuntimeWarning: Expected to find a single channels file associated with sub-001_ses-N170_task-N170, but found 2: "['F:\\Masterthesis\\Data\\N170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_badChannels.tsv', 'F:\\Masterthesis\\Data\\N170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_channels.tsv']".

The search_str was "F:\Masterthesis\Data\N170\sub-001\**\sub-001_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-59-03cda6a932f4>:7: RuntimeWarning: Participants file not found for sub-001_ses-N170_task-N170_eeg.set... Not reading in any particpants.tsv data.
  raw = read_raw_bids(bids_path)

Creating augmented epochs: 100%|█████████████████████████████████████████